In [140]:
import random

numbers_list = []
last = 0
for i in range(10_000):
    last += random.randint(1, 10)
    numbers_list += [last]

numbers_list[0:5]

[10, 20, 23, 24, 26]

In [74]:
import math


def btree_organize(numbers_list):
    if len(numbers_list) < 10:
        return numbers_list 

    middle = numbers_list[math.floor(len(numbers_list) / 2) - 1]
    smaller = [x for x in numbers_list if x < middle]
    bigger = [x for x in numbers_list if x > middle]
    return { "middle": middle, "left": btree_organize(smaller), "right": btree_organize(bigger) }


numbers_btree = btree_organize(numbers_list)
str(numbers_btree)[0:400] + "..."

"{'middle': 27955, 'left': {'middle': 14041, 'left': {'middle': 7083, 'left': {'middle': 3539, 'left': {'middle': 1727, 'left': {'middle': 833, 'left': {'middle': 419, 'left': {'middle': 194, 'left': {'middle': 85, 'left': {'middle': 40, 'left': [7, 10, 11, 16, 26, 30, 34], 'right': [42, 50, 51, 61, 67, 74, 76, 81, 83]}, 'right': {'middle': 149, 'left': [94, 96, 104, 112, 119, 126, 135, 139], 'righ..."

In [77]:
def btree_search(number, numbers_btree, steps=0):
    if 'middle' in numbers_btree:
        if number < numbers_btree['middle']:
            return btree_search(number, numbers_btree['left'], steps + 1)
        elif number > numbers_btree['middle']:
            return btree_search(number, numbers_btree['right'], steps + 1)
        else:
            return { "steps": steps, "found": numbers_btree['middle'] }
    else:
        if number in numbers_btree:
            steps += numbers_btree.index(number) + 1
            return { "steps": steps, "found": number }
        else:
            return { "steps": steps, "found": None }
        
btree_search(42, numbers_btree)

{'steps': 11, 'found': 42}

In [97]:
from statistics import mean

sample = random.sample(numbers_list, 1000)

%time steps = [btree_search(s, numbers_btree)['steps'] for s in sample]
print("Avg number of btree steps", mean(steps))
print("\n")
%time indexes = [numbers_list.index(s) for s in sample]
print("Avg number of indexes", mean(indexes))

CPU times: user 12.4 ms, sys: 1.29 ms, total: 13.7 ms
Wall time: 12.5 ms
Avg number of btree steps 13.862


CPU times: user 220 ms, sys: 4.92 ms, total: 225 ms
Wall time: 235 ms
Avg number of indexes 5080.308


In [112]:
import numpy as np
from sklearn.linear_model import LinearRegression

X = np.array(numbers_list).reshape(-1, 1)
y = np.array(range(0, len(numbers_list)))

model = LinearRegression().fit(X, y)
model.score(X, y)

0.999955718673992

In [129]:
def predict_index(number, model):
    return math.ceil(model.predict([[number]])[0])


print("Predictect index", predict_index(55382, model), "Actual index", numbers_list.index(55382))

Predictect index 9957 Actual index 9973


In [136]:
def regression_search(number, numbers_list, model, steps=0):
    maximum = len(numbers_list) - 1
    predicted_index = min(predict_index(number, model), maximum)
    steps += 1
    if numbers_list[predicted_index] == number:
        return { "steps": steps, "found": number }
    
    right = 1
    left = 1
    while True:
        if numbers_list[predicted_index + right] == number:
            return { "steps": steps + 1, "found": number }
        elif numbers_list[predicted_index - left] == number:
            return { "steps": steps + 1, "found": number }
        else:
            right += 1
            left += 1
            steps += 2
    
regression_search(55382, numbers_list, model)

{'steps': 32, 'found': 55382}

In [138]:
sample = random.sample(numbers_list, 1000)

%time steps = [btree_search(s, numbers_btree)['steps'] for s in sample]
print("Avg number of btree steps", mean(steps))
print("\n")

%time steps = [regression_search(s, numbers_list, model)['steps'] for s in sample]
print("Avg number of regression steps", mean(steps))
print("\n")

%time indexes = [numbers_list.index(s) for s in sample]
print("Avg number of indexes", mean(indexes))

CPU times: user 16.2 ms, sys: 1.98 ms, total: 18.1 ms
Wall time: 19.1 ms
Avg number of btree steps 13.786


CPU times: user 168 ms, sys: 7.17 ms, total: 175 ms
Wall time: 228 ms
Avg number of regression steps 30.762


CPU times: user 208 ms, sys: 4.41 ms, total: 212 ms
Wall time: 246 ms
Avg number of indexes 4896.157
